In [1]:
# Install dependencies (run once per environment)
%pip install -q dspy python-dotenv requests beautifulsoup4 lxml pandas


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Basic imports and environment setup
import os
import json
import time
from datetime import datetime, timezone
from pathlib import Path
from urllib.parse import urljoin, urlparse

import dspy
import requests
from bs4 import BeautifulSoup
import io
import pandas as pd
from dotenv import load_dotenv
import logging
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FuturesTimeoutError

# Load .env for API keys and SMTP configuration
load_dotenv()

# Configure DSPy LM (align with other notebooks)
lm = dspy.LM(
    "openai/gpt-5-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=1,
    max_tokens=16000,
)

dspy.configure(lm=lm)

# Logging and timeouts
LOG_LEVEL = os.getenv("COMP_MONITOR_LOG_LEVEL", "INFO").upper()
logging.basicConfig(
    level=getattr(logging, LOG_LEVEL, logging.INFO),
    format="%(asctime)s %(levelname)s %(message)s",
)
logger = logging.getLogger("competitor_monitor")

HTTP_TIMEOUT_SEC = int(os.getenv("COMP_MONITOR_HTTP_TIMEOUT", "20"))
LM_TIMEOUT_SEC = int(os.getenv("COMP_MONITOR_LM_TIMEOUT", "60"))
MAX_SPR_PAGES_DEFAULT = int(os.getenv("COMP_MONITOR_MAX_SPR_PAGES", "25"))
MAX_LINKS_PER_SITE = int(os.getenv("COMP_MONITOR_MAX_LINKS_PER_SITE", "5"))

# HTTP defaults
DEFAULT_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
}

# Simple in-repo persistence for URL diffing
STATE_DIR = Path("state")
STATE_DIR.mkdir(exist_ok=True)
URL_STORE_PATH = STATE_DIR / "competitor_urls.json"

# Global working state
monitor_state = {
    "competitors": [],            # rows parsed from CSV
    "discovered_urls": [],        # all discovered links (normalized, in-domain)
    "new_urls": [],               # links not seen before, persisted with date_found
    "spr_items": [],              # [{source_url,title,date,spr:[...]}, ...]
    "item_summaries": [],         # [str]
    "executive_summary": "",     # str
    "email_html": "",            # str
}

logger.info("DSPy configured. Competitor monitoring agent ready.")
print("DSPy configured. Competitor monitoring agent ready.")


2025-10-10 18:04:06,100 INFO DSPy configured. Competitor monitoring agent ready.


DSPy configured. Competitor monitoring agent ready.


In [3]:
# Helper functions: CSV fetch, HTML parsing, URL normalization, and persistence

def run_with_timeout(func, timeout_sec: int, label: str):
    start = time.perf_counter()
    executor = ThreadPoolExecutor(max_workers=1)
    future = executor.submit(func)
    try:
        result = future.result(timeout=timeout_sec)
        elapsed = time.perf_counter() - start
        logger.info(f"{label} completed in {elapsed:.2f}s")
        return result
    except FuturesTimeoutError:
        elapsed = time.perf_counter() - start
        logger.warning(f"{label} timed out after {elapsed:.2f}s (limit {timeout_sec}s)")
        # Best-effort cancel; underlying operation may continue in background
        future.cancel()
        raise
    finally:
        # Return control immediately; don't wait for unfinished tasks
        try:
            executor.shutdown(wait=False, cancel_futures=True)
        except TypeError:
            executor.shutdown(wait=False)


def read_competitor_csv(csv_url: str) -> list[dict]:
    """
    Download a published CSV of competitors. Expected columns:
    - 'Url': canonical domain (e.g., https://example.com)
    - 'blog urls': blog or news root (e.g., https://example.com/blog)
    Returns a list of row dicts.
    """
    t0 = time.perf_counter()
    logger.info(f"Fetching competitors CSV: {csv_url}")
    resp = requests.get(csv_url, timeout=HTTP_TIMEOUT_SEC)
    resp.raise_for_status()
    df = pd.read_csv(io.StringIO(resp.text))
    # Keep only expected columns if present
    rows = []
    for _, r in df.iterrows():
        row = {k: (str(r[k]).strip() if k in r and pd.notna(r[k]) else "") for k in ["Url", "blog urls"]}
        if row["Url"] and row["blog urls"]:
            rows.append(row)
    logger.info(f"CSV parsed: {len(rows)} rows in {time.perf_counter()-t0:.2f}s")
    return rows


def fetch_html(url: str, headers: dict | None = None, max_redirects: int = 2) -> str:
    """
    Minimal HTTP GET for HTML content. Returns response text or raises.
    """
    t0 = time.perf_counter()
    session = requests.Session()
    session.max_redirects = max_redirects
    logger.info(f"HTTP GET {url}")
    resp = session.get(url, headers=headers or DEFAULT_HEADERS, timeout=HTTP_TIMEOUT_SEC)
    resp.raise_for_status()
    resp.encoding = resp.encoding or 'utf-8'
    html = resp.text
    logger.info(f"HTTP {url} -> {len(html)} bytes in {time.perf_counter()-t0:.2f}s")
    return html


def extract_links(html: str, base_url: str) -> list[str]:
    """
    Extract absolute links from HTML, normalized against base_url.
    Preserves document order and strips URL fragments.
    """
    soup = BeautifulSoup(html, "lxml")
    links_ordered: list[str] = []
    seen: set[str] = set()
    for a in soup.find_all("a", href=True):
        href = a["href"].strip()
        if not href:
            continue
        abs_url = urljoin(base_url, href)
        parsed = urlparse(abs_url)
        if parsed.scheme in {"http", "https"}:
            cleaned = parsed._replace(fragment="").geturl()
            if cleaned not in seen:
                seen.add(cleaned)
                links_ordered.append(cleaned)
    logger.info(f"Extracted {len(links_ordered)} links from {base_url}")
    return links_ordered


def same_host(url_a: str, url_b: str) -> bool:
    """
    True if url_a and url_b share the same netloc (host:port), ignoring scheme.
    """
    pa, pb = urlparse(url_a), urlparse(url_b)
    return pa.netloc.lower() == pb.netloc.lower()


def filter_articleish_links(links: list[str]) -> list[str]:
    """
    Keep links that look like articles/posts and drop assets, auth, feeds, and anchors.
    Heuristics are simple and conservative.
    """
    drop_substrings = [
        "#", "?share=", "?utm_", "/tag/", "/category/", "/topics/", "/author/",
        "/login", "/signin", "/signup", "/register", "/account", "/privacy", "/terms",
        "/feed", "/rss", "/atom", ".xml",
    ]
    drop_extensions = {
        ".jpg", ".jpeg", ".png", ".gif", ".webp", ".svg", ".ico",
        ".css", ".js", ".pdf", ".zip", ".mp4", ".mov", ".avi", ".woff", ".woff2", ".ttf",
    }
    kept: list[str] = []
    for url in links:
        lower = url.lower()
        if any(s in lower for s in drop_substrings):
            continue
        path = urlparse(lower).path
        if any(path.endswith(ext) for ext in drop_extensions):
            continue
        kept.append(url)
    return kept


def load_url_store() -> dict:
    """
    Load persisted URL store from disk. Format:
    {
      "urls": {
        "https://example.com/post": {"source_site": "https://example.com", "date_found": "ISO8601"}
      }
    }
    """
    t0 = time.perf_counter()
    if URL_STORE_PATH.exists():
        with URL_STORE_PATH.open("r", encoding="utf-8") as f:
            data = json.load(f)
        logger.info(f"Loaded URL store with {len(data.get('urls', {}))} entries in {time.perf_counter()-t0:.2f}s")
        return data
    logger.info("URL store not found; starting fresh")
    return {"urls": {}}


def save_url_store(store: dict) -> None:
    t0 = time.perf_counter()
    URL_STORE_PATH.parent.mkdir(parents=True, exist_ok=True)
    with URL_STORE_PATH.open("w", encoding="utf-8") as f:
        json.dump(store, f, indent=2)
    logger.info(f"Saved URL store with {len(store.get('urls', {}))} entries in {time.perf_counter()-t0:.2f}s")


def upsert_new_urls(discovered: list[dict]) -> list[dict]:
    """
    Persist newly discovered URLs, returning only new ones.
    Each discovered item: {"normalized_url": str, "source_site": str}
    """
    t0 = time.perf_counter()
    store = load_url_store()
    known = store.get("urls", {})
    new_items: list[dict] = []
    for item in discovered:
        url = item["normalized_url"]
        if url not in known:
            known[url] = {
                "source_site": item.get("source_site", ""),
                "date_found": datetime.now(timezone.utc).isoformat(),
            }
            new_items.append({
                "normalized_url": url,
                "source_site": item.get("source_site", ""),
                "date_found": known[url]["date_found"],
            })
    store["urls"] = known
    save_url_store(store)
    logger.info(f"Discovered {len(discovered)} urls; {len(new_items)} new in {time.perf_counter()-t0:.2f}s")
    return new_items

logger.info("Helpers ready: CSV, HTTP, HTML, normalization, persistence.")
print("Helpers ready: CSV, HTTP, HTML, normalization, persistence.")


2025-10-10 18:06:18,989 INFO Helpers ready: CSV, HTTP, HTML, normalization, persistence.


Helpers ready: CSV, HTTP, HTML, normalization, persistence.


In [4]:
# DSPy signatures and predictors

class SPRSignature(dspy.Signature):
    """
    Render page content as a Sparse Priming Representation (SPR) for downstream LLM use.
    Output JSON array with: source_url, title, date, spr (list of statements).
    """
    source_url: str = dspy.InputField()
    page_text: str = dspy.InputField()
    title_hint: str = dspy.InputField()
    spr_json: str = dspy.OutputField(description="JSON array string as specified")


class ItemSummarySignature(dspy.Signature):
    """
    Summarize a single competitor page based on its SPR into 2–3 sentences.
    Audience: business owner. Reference the source and label competitor.
    """
    competitor_name: str = dspy.InputField()
    source_url: str = dspy.InputField()
    spr_json: str = dspy.InputField()
    summary: str = dspy.OutputField()


class ExecSummarySignature(dspy.Signature):
    """
    Aggregate multiple item summaries and write a concise 2–3 sentence
    executive summary highlighting trends, themes, and outliers. Output JSON only.
    """
    summaries_json: str = dspy.InputField()
    executive_summary_json: str = dspy.OutputField()


spr_predict = dspy.Predict(SPRSignature)
item_summary_predict = dspy.Predict(ItemSummarySignature)
exec_summary_predict = dspy.Predict(ExecSummarySignature)

print("Signatures and predictors ready.")


Signatures and predictors ready.


In [5]:
# Tools: mirror n8n nodes as callable functions returning dicts for agent state mgmt

# 1) Get competitors CSV

def tool_get_competitors(csv_url: str) -> dict:
    t0 = time.perf_counter()
    logger.info(f"tool_get_competitors: start csv_url={csv_url}")
    rows = read_competitor_csv(csv_url)
    monitor_state["competitors"] = rows
    logger.info(f"tool_get_competitors: loaded {len(rows)} competitors in {time.perf_counter()-t0:.2f}s")
    return {"tool": "get_competitors", "count": len(rows)}


# 2) Discover new URLs from each competitor blog root

def tool_discover_new_urls() -> dict:
    t0 = time.perf_counter()
    discovered: list[dict] = []
    competitors = monitor_state.get("competitors", [])
    logger.info(f"tool_discover_new_urls: start for {len(competitors)} competitors")
    for row in competitors:
        site = row["Url"].strip()
        blog_root = row["blog urls"].strip()
        try:
            logger.info(f"Discovering links from {blog_root} (site {site})")
            html = fetch_html(blog_root)
            links = extract_links(html, blog_root)
            # Filter to same-host and article-ish
            same_host_links = [u for u in links if same_host(u, site) or same_host(u, blog_root)]
            filtered = filter_articleish_links(same_host_links)
            # Heuristic: most recent links tend to appear later or earlier depending on theme.
            # To be robust, take the last N unique in document order (preserved) which often correspond to recent posts widgets.
            limited = filtered[-MAX_LINKS_PER_SITE:] if MAX_LINKS_PER_SITE > 0 else filtered
            logger.info(
                f"{blog_root}: extracted={len(links)} same_host={len(same_host_links)} articleish={len(filtered)} kept={len(limited)}"
            )
            for link in limited:
                discovered.append({
                    "normalized_url": link,
                    "source_site": site,
                })
        except Exception as e:
            logger.warning(f"Discovery failed for {blog_root}: {e}")
            continue
    # de-duplicate by URL while preserving last occurrence (recent-first bias)
    unique: dict[str, dict] = {}
    for item in discovered:
        unique[item["normalized_url"]] = item
    discovered_list = list(unique.values())
    monitor_state["discovered_urls"] = discovered_list
    logger.info(
        f"tool_discover_new_urls: found {len(discovered)} links, {len(discovered_list)} unique in {time.perf_counter()-t0:.2f}s"
    )

    # Persist and compute diff
    new_items = upsert_new_urls(discovered_list)
    monitor_state["new_urls"] = new_items
    logger.info(f"tool_discover_new_urls: {len(new_items)} new URLs")
    return {"tool": "discover_new_urls", "new_count": len(new_items)}


# 3) Generate SPR for each new URL

def tool_generate_spr(max_pages: int = MAX_SPR_PAGES_DEFAULT) -> dict:
    t0 = time.perf_counter()
    spr_items: list[dict] = []
    urls = monitor_state.get("new_urls", [])[:max_pages]
    logger.info(f"tool_generate_spr: start for {len(urls)} pages (cap={max_pages})")
    for item in urls:
        url = item["normalized_url"]
        try:
            html = fetch_html(url)
            soup = BeautifulSoup(html, "lxml")
            title = soup.title.string.strip() if soup.title and soup.title.string else "unknown"
            # Simple text extraction
            for s in soup(["script", "style", "noscript"]):
                s.extract()
            text = " ".join(soup.get_text(separator=" ").split())[:20000]

            logger.info(f"SPR predict start: {url} (text_len={len(text)})")
            pred = run_with_timeout(
                lambda: spr_predict(source_url=url, page_text=text, title_hint=title),
                LM_TIMEOUT_SEC,
                f"SPR predict {url}",
            )
            spr_items.append({
                "source_url": url,
                "title": title,
                "spr_json": pred.spr_json,
                "date_found": item.get("date_found", "unknown"),
            })
        except FuturesTimeoutError:
            logger.warning(f"SPR timeout for {url}")
            continue
        except Exception as e:
            logger.warning(f"SPR failed for {url}: {e}")
            continue
    monitor_state["spr_items"] = spr_items
    logger.info(f"tool_generate_spr: completed {len(spr_items)} items in {time.perf_counter()-t0:.2f}s")
    return {"tool": "generate_spr", "count": len(spr_items)}


# 4) Summarize items (per-URL summaries)

def tool_summarize_items() -> dict:
    t0 = time.perf_counter()
    summaries: list[str] = []
    for item in monitor_state.get("spr_items", []):
        source = item["source_url"]
        comp = urlparse(item.get("source_url", "")).netloc
        try:
            logger.info(f"Item summary start: {source}")
            pred = run_with_timeout(
                lambda: item_summary_predict(
                    competitor_name=comp,
                    source_url=source,
                    spr_json=item["spr_json"],
                ),
                LM_TIMEOUT_SEC,
                f"Item summary {source}",
            )
            summaries.append(pred.summary)
        except FuturesTimeoutError:
            logger.warning(f"Item summary timeout: {source}")
            continue
        except Exception as e:
            logger.warning(f"Item summary failed for {source}: {e}")
            continue
    monitor_state["item_summaries"] = summaries
    logger.info(f"tool_summarize_items: {len(summaries)} summaries in {time.perf_counter()-t0:.2f}s")
    return {"tool": "summarize_items", "count": len(summaries)}


# 5) Executive summary

def tool_exec_summary() -> dict:
    t0 = time.perf_counter()
    # wrap summaries as JSON array for the signature
    payload = json.dumps(monitor_state.get("item_summaries", []))
    try:
        pred = run_with_timeout(
            lambda: exec_summary_predict(summaries_json=payload),
            LM_TIMEOUT_SEC,
            "Exec summary",
        )
        # Expect JSON string in output
        monitor_state["executive_summary"] = pred.executive_summary_json
    except FuturesTimeoutError:
        logger.warning("Executive summary timeout; leaving as empty string")
        monitor_state["executive_summary"] = "{}"
    except Exception as e:
        logger.warning(f"Executive summary failed: {e}")
        monitor_state["executive_summary"] = "{}"
    logger.info(f"tool_exec_summary: done in {time.perf_counter()-t0:.2f}s")
    return {"tool": "exec_summary"}


# 6) Build email HTML (no sending here)

def tool_build_email_html(to_email: str, subject: str = "daily competitor digest") -> dict:
    logger.info("tool_build_email_html: start")
    exec_summary_json = monitor_state.get("executive_summary", "{}")
    try:
        exec_text = json.loads(exec_summary_json)
        if isinstance(exec_text, dict) and "summary" in exec_text:
            exec_html = exec_text["summary"]
        elif isinstance(exec_text, str):
            exec_html = exec_text
        else:
            exec_html = json.dumps(exec_text)
    except Exception:
        exec_html = exec_summary_json

    items = monitor_state.get("item_summaries", [])
    items_html = "".join([f"<li>{s}</li>" for s in items])

    html = f"""
    <p>Hi,</p>
    <p>Here’s the daily scoop:</p>
    <h3>Executive Summary</h3>
    <p>{exec_html}</p>
    <h3>Competitor Updates</h3>
    <ul>
    {items_html}
    </ul>
    <p>Have a great day,</p>
    <hr>
    <p><em>This email was sent automatically by the DSPy agent</em></p>
    """.strip()

    monitor_state["email_html"] = html
    logger.info(f"tool_build_email_html: built html length={len(html)} items={len(items)}")
    return {"tool": "build_email_html", "to": to_email, "subject": subject}


# 7) Send email via SMTP (optional; stub to avoid side-effects)

def tool_send_email_smtp(to_email: str, subject: str) -> dict:
    """
    Minimal SMTP stub to avoid side effects. Implement with real SMTP creds if needed.
    """
    logger.info(f"tool_send_email_smtp: stub send to={to_email} subject={subject}")
    # In this repo we avoid actually sending email; return payload for inspection
    return {
        "tool": "send_email_smtp",
        "to": to_email,
        "subject": subject,
        "html_length": len(monitor_state.get("email_html", "")),
    }

logger.info("Tools ready: get_competitors, discover_new_urls, generate_spr, summarize_items, exec_summary, build_email_html, send_email_smtp")
print("Tools ready: get_competitors, discover_new_urls, generate_spr, summarize_items, exec_summary, build_email_html, send_email_smtp")


2025-10-10 18:10:05,257 INFO Tools ready: get_competitors, discover_new_urls, generate_spr, summarize_items, exec_summary, build_email_html, send_email_smtp


Tools ready: get_competitors, discover_new_urls, generate_spr, summarize_items, exec_summary, build_email_html, send_email_smtp


In [6]:
# ReAct agent definition

class CompetitorAgentSignature(dspy.Signature):
    """
    Competitor Monitoring Agent
    Tasks:
    - Fetch competitors CSV
    - Crawl blog roots, discover in-domain links, diff against store
    - For new URLs: generate SPR, write per-item summaries
    - Aggregate an executive summary
    - Build email HTML (and optionally send)
    Return final HTML in `email_html_out`.
    """
    csv_url: str = dspy.InputField()
    to_email: str = dspy.InputField()
    reasoning: str = dspy.OutputField()
    email_html_out: str = dspy.OutputField()


competitor_agent = dspy.ReAct(
    CompetitorAgentSignature,
    tools=[
        tool_get_competitors,
        tool_discover_new_urls,
        tool_generate_spr,
        tool_summarize_items,
        tool_exec_summary,
        tool_build_email_html,
        tool_send_email_smtp,
    ],
)

logger.info("ReAct agent ready.")
print("ReAct agent ready.")


2025-10-10 18:10:26,846 INFO ReAct agent ready.


ReAct agent ready.


In [7]:
# Demo: run the agent
CSV_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTvE_qRDWRu5hjZ45yY6juAc4i7iT3DIrJT9q3cz29uIpGpz0IRRzHPdqcKge8obrTjwNS7qC3TGGg-/pub?gid=0&single=true&output=csv"
TO_EMAIL = os.getenv("COMP_MONITOR_TO_EMAIL", "youremailhere")

# Clear working state for demo
monitor_state["competitors"] = []
monitor_state["discovered_urls"] = []
monitor_state["new_urls"] = []
monitor_state["spr_items"] = []
monitor_state["item_summaries"] = []
monitor_state["executive_summary"] = ""
monitor_state["email_html"] = ""

logger.info(f"Demo start: csv={CSV_URL} to={TO_EMAIL}")
t0 = time.perf_counter()
try:
    result = competitor_agent(csv_url=CSV_URL, to_email=TO_EMAIL)
    logger.info(f"Agent run completed in {time.perf_counter()-t0:.2f}s")
except Exception as e:
    logger.exception(f"Agent run failed: {e}")
    raise

# Build email HTML via tool explicitly to ensure output string
tool_build_email_html(to_email=TO_EMAIL, subject="daily competitor digest")

logger.info(
    f"Run summary: new_urls={len(monitor_state['new_urls'])} items={len(monitor_state['item_summaries'])} html_len={len(monitor_state['email_html'])}"
)

print("Reasoning:\n", result.reasoning)
print("\nNew URLs:", len(monitor_state["new_urls"]))
print("Items summarized:", len(monitor_state["item_summaries"]))
print("\nExecutive summary (JSON):\n", monitor_state["executive_summary"]) 
print("\nEmail preview (first 800 chars):\n", monitor_state["email_html"][:800])


2025-10-10 18:10:31,174 INFO Demo start: csv=https://docs.google.com/spreadsheets/d/e/2PACX-1vTvE_qRDWRu5hjZ45yY6juAc4i7iT3DIrJT9q3cz29uIpGpz0IRRzHPdqcKge8obrTjwNS7qC3TGGg-/pub?gid=0&single=true&output=csv to=youremailhere
2025-10-10 18:10:36,735 INFO tool_get_competitors: start csv_url=https://docs.google.com/spreadsheets/d/e/2PACX-1vTvE_qRDWRu5hjZ45yY6juAc4i7iT3DIrJT9q3cz29uIpGpz0IRRzHPdqcKge8obrTjwNS7qC3TGGg-/pub?gid=0&single=true&output=csv
2025-10-10 18:10:36,736 INFO Fetching competitors CSV: https://docs.google.com/spreadsheets/d/e/2PACX-1vTvE_qRDWRu5hjZ45yY6juAc4i7iT3DIrJT9q3cz29uIpGpz0IRRzHPdqcKge8obrTjwNS7qC3TGGg-/pub?gid=0&single=true&output=csv
2025-10-10 18:10:37,941 INFO CSV parsed: 1 rows in 1.21s
2025-10-10 18:10:37,943 INFO tool_get_competitors: loaded 1 competitors in 1.21s
2025-10-10 18:10:42,580 INFO tool_discover_new_urls: start for 1 competitors
2025-10-10 18:10:42,580 INFO Discovering links from https://askrally.com/articles (site https://askrally.com)
2025-10-10

Reasoning:
 I fetched the competitors CSV (1 competitor) and crawled the listed blog root(s). I discovered in-domain links and compared them to the existing store of known URLs. No new URLs were found (new_count = 0), so no SPRs or per-item summaries were generated.

Summary of actions:
- Fetched competitors CSV (source provided).
- Crawled blog root(s) for 1 competitor.
- Discovered in-domain links and diffed against the store.
- New URLs found: 0 — no new SPRs or summaries created.
- Executive summary prepared.

Recommended next steps:
- Continue daily monitoring; the pipeline is working and found no new content today.
- If you want broader coverage, consider including sitemaps, RSS feeds, or deeper crawl depths.
- If you want alerts for specific content types (product mentions, pricing, promos), I can add keyword filters and escalation rules.
- If you want the HTML emailed automatically on each run, I can enable a send step.

Date of run: 2025-10-10

New URLs: 0
Items summarized: 0
